___
# Ciência dos Dados - PROJETO 3

___
### *Bruno Kaczelnik, Guilherme Lotaif, Renato Yasuo Chopard Tajima, Thiago Verardo*
___

# Análise de Airbnb nas maiores cidades americanas

___

## 1. Introdução

   Neste Projeto será feita uma análise dos dados que possuimos de Airbnb nos Estados Unidos, e a partir dos dados que julgarmos pertinentes ao nosso estudo. Serão usados diferentes **métodos de predição** para descobrir`o preço de um aluguel dependendo das características fornecidas`, após usarmos os diferentes métodos, vamos comparar os resultados de cada método, finalmente poderemos concluir o projeto..

   Um pouco mais sobre a plataforma digital Airbnb: Ela é utilizada para efetuar o aluguel de uma casa ou apartamento de outros usuários da própria plataforma. Com anúncios em 192 países, podemos concluir que é uma plataforma é bem grande, somente nos Estados Unidos existem aproximadamente 600.000 possiveis locais para aluguel. Com tantas opções, a escolha que o usuário faz quanto ao local é feita baseada nas **informações fornecidas pelo propretario**, tais informações podem variar desde quantos banheiros a residência possui, até se ela possui acesso a uma rede Wifi.

**Fonte do dataset:** www.kaggle.com/rudymizrahi/airbnb-listings-in-major-us-cities-deloitte-ml

___
## 2. Minerando dados e características do dataset

O dataset que vamos utilizar nesse projeto possui diversas colunas com informações dos Airbnb nos Estados Unidos, portannto temos que percorrer todas essas colunas para fazer uma limpeza, e deixar somente as informações que serão pertinentes a nossa análise.

**Importando as bibliotecas que serão utilizadas:**

In [18]:
#Importando bibliotecas:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import pandas_profiling
import matplotlib.pyplot as plt
from scipy.stats import norm,probplot

plt.style.use('ggplot')

In [19]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/guilherme/Downloads/Projeto-3-CDD---DPA-master


### - Base de dados

In [20]:
#Importado o arquivo de treinamento:
df_train = pd.read_csv('train.csv')

#Importado o arquivo de treinamento:
df_test = pd.read_csv('test.csv')

In [21]:
#Análisando o tamanho do dataframe de treino:
linhas, colunas = df_train.shape
print("O Dataframe possui {0} linhas por {1} colunas.".format(linhas, colunas))

#Análisando o tamanho do dataframe teste:
linhas, colunas = df_test.shape
print("O Dataframe possui {0} linhas por {1} colunas.".format(linhas, colunas))

O Dataframe possui 74111 linhas por 29 colunas.
O Dataframe possui 25458 linhas por 28 colunas.


...

#### Limpando o dataframe de treino para ser análisado:
Esta etapa consiste em uma preparação do dataframe para facilitar a análise no mesmo, assim evitando ocorrências de complicações ou erros que atrapalhem futuramento os nosssos classificadores. Portanto iremos limpar os titulos de cada coluna, vamos remover os valores nulos de cada categoria, assim como colunas desnecessárias.

In [22]:
#Removendo os espaços em branco dos nomes das colunas:
df_train.columns = [espaços.strip() for espaços in df_train.columns.tolist()]

In [23]:
#Removendo colunas que não sao pertinentes ao estudo:
df_train = df_train.drop(["latitude","longitude","name","thumbnail_url","id","description"],axis=1);

In [24]:
#Apagando valores nulos que podem causar problemas posteriores:
df_train = df_train.dropna(axis=0, subset=['bathrooms','first_review','host_has_profile_pic','host_identity_verified',
                               'host_response_rate','host_since','last_review','neighbourhood','review_scores_rating',
                               'zipcode','bedrooms','beds'])

In [25]:
#Corrigindo simplificação no arquivo:
df_train.loc[(df_train["instant_bookable"] == "f"),"instant_bookable"] = "False"
df_train.loc[(df_train["instant_bookable"] == "t"),"instant_bookable"] = "True"

df_train.loc[(df_train["host_has_profile_pic"] == "f"),"host_has_profile_pic"] = "False"
df_train.loc[(df_train["host_has_profile_pic"] == "t"),"host_has_profile_pic"] = "True"


df_train.loc[(df_train["host_identity_verified"] == "f"),"host_identity_verified"] = "False"
df_train.loc[(df_train["host_identity_verified"] == "t"),"host_identity_verified"] = "True"

In [26]:
df_train.head(2)

,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,...,host_response_rate,host_since,instant_bookable,last_review,neighbourhood,number_of_reviews,review_scores_rating,zipcode,bedrooms,beds
1,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,...,100%,2017-06-19,True,2017-09-23,Hell's Kitchen,6,93.0,10019,3.0,3.0
2,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,...,100%,2016-10-25,True,2017-09-14,Harlem,10,92.0,10027,1.0,3.0


...

#### Limpando o dataframe de teste para ser análisado:
Esta etapa consiste em uma preparação do dataframe para facilitar a análise no mesmo, assim evitando ocorrências de complicações ou erros que atrapalhem futuramento os nosssos classificadores. Portanto iremos limpar os titulos de cada coluna, vamos remover os valores nulos de cada categoria, assim como colunas desnecessárias.

In [27]:
#Removendo os espaços em branco dos nomes das colunas:
df_test.columns = [espaços.strip() for espaços in df_test.columns.tolist()]

In [28]:
#Removendo colunas que não sao pertinentes ao estudo:
df_test = df_test.drop(["latitude","longitude","name","thumbnail_url","id","description"],axis=1);

In [29]:
#Apagando valores nulos que podem causar problemas posteriores:
df_test = df_test.dropna(axis=0, subset=['bathrooms','first_review','host_has_profile_pic','host_identity_verified',
                               'host_response_rate','host_since','last_review','neighbourhood','review_scores_rating',
                               'zipcode','bedrooms','beds'])

In [30]:
#Corrigindo simplificação no arquivo:
df_test.loc[(df_test["instant_bookable"] == "f"),"instant_bookable"] = "False"
df_test.loc[(df_test["instant_bookable"] == "t"),"instant_bookable"] = "True"

df_test.loc[(df_test["host_has_profile_pic"] == "f"),"host_has_profile_pic"] = "False"
df_test.loc[(df_test["host_has_profile_pic"] == "t"),"host_has_profile_pic"] = "True"


df_test.loc[(df_test["host_identity_verified"] == "f"),"host_identity_verified"] = "False"
df_test.loc[(df_test["host_identity_verified"] == "t"),"host_identity_verified"] = "True"

In [31]:
df_test.sample(2)

,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,first_review,...,host_response_rate,host_since,instant_bookable,last_review,neighbourhood,number_of_reviews,review_scores_rating,zipcode,bedrooms,beds
19379,House,Private room,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,2.0,Real Bed,flexible,True,DC,2017-01-22,...,100%,2014-05-23,False,2017-03-05,Logan Circle,4,90.0,20005,1.0,1.0
24382,Apartment,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,1.0,Real Bed,moderate,True,NYC,2013-09-07,...,100%,2012-05-27,False,2014-04-02,Boerum Hill,10,95.0,11201,1.0,1.0


### CHECAR DESCRIPTION

#### Separando os atributos na coluna de características:
<br>
Para um melhor rendimento dos métodos de prdição, vamos separar cada atributo para que eles possam ser análisados e comparados de maneira unitária, e para isso vamos juntar todas palavras em listas para cada anúncio, e depois comparar não só os itens quanto o tamanho das listas também.

In [ ]:
'''#Criando uma lista para as palavras dos emails SPAM:
dic_amenities = {}
#Criando a variável contador para o total de palavras SPAM:
contador_amenities = 0

#Criando um loop para atribuir os emails SPAM a uma variável:
for qualidades in df.amenities:
    #Criando um loop para alocar as palavras de SPAM no dicionário:
    for caracteristica in qualidades:
        #print(caracteristica)
        if caracteristica not in dic_amenities:
            print(caracteristica)
            dic_amenities[caracteristica] =1
            contador_amenities +=1
        else:
            dic_amenities[caracteristica] +=1
            contador_amenities +='''

<br>

### - Análise descritiva

Após realizarmos o filtro, deve-se realizar uma análise exploratória dos dados, com o  objetivo de achar as váriaveis que mais influenciam no nosso objetivo e que assim possam nos ajudar a prever qual será a avaliação de um hotel aleatório. Ela será feita com o auxílio do pandas_profiling e seaborn.


In [ ]:
#utilizandoo o pandas_profiling
#df é o dataframe após o filtro
pandas_profiling.ProfileReport(df_train)

___
## 3. Modelos de predição

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
#Removendo colunas que não sao pertinentes ao estudo:
df2 = df.drop(["latitude","longitude","name","thumbnail_url","id","description"],axis=1);

In [ ]:
#Dividindo os dados do dataframe em teste e treino:
X_train, X_test, y_train, y_test = train_test_split(df1, df2['log_price'], test_size = 0.33, random_state = 0)                           

#Chamando o classificador:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, 
                                   max_depth=8, min_samples_leaf=4)
#Fazendo um fit nos dados de treino:
clf_gini.fit(X_train, y_train)   

y_pred = clf_gini.predict(X_test)

#Descobrindo a acurácia do classificador:
acc = accuracy_score(y_test, y_pred)

acc = acc*100
print("A acurácia é de {:.2f}%".format(acc))

<font color=red>o MODELO DO RANDOM FOREST (Para comparação)</font>

___
## 4. Processo e estatísticas de validação

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## 5. Conclusão

___
## 6. Referências bibliográficas